In [8]:
import pdal
import json
import geopandas as gpd
import pandas as pd

In [18]:
pipeline = [
        {
            "type": "readers.las",
            "filename": f'data/sample/MDS_color_3316-221.laz',
            "override_srs": "EPSG:31983"
        },
        {
            "type":"filters.range",
            "limits":"ReturnNumber[1:4],NumberOfReturns![1:1],UserData[3:],Classification[5:5]"
        },
        {
            "type": "filters.assign",
            "value" : "Z = 1"
        },
        {
            "filename":f"results/sample/MDS_3316-221-vegetation.tiff",
            "gdaldriver":"GTiff",
            "output_type":"max",
            "resolution":"1",
            "type": "writers.gdal",
            "gdalopts":"COMPRESS=ZSTD, PREDICTOR=3, BIGTIFF=YES",
            # "width": scm_att.width,
            # "height": scm_att.height,
            # "origin_x": scm_att.origin_x,
            # "origin_y": scm_att.origin_y,
            "nodata":"0",
            "data_type": "float32",
            "where": "(Classification == 5)",
            "default_srs": "EPSG:31983"
        },
        
    ]

In [19]:
filtered = pdal.Pipeline(json.dumps(pipeline))
n_points = filtered.execute()

In [21]:
!gdal_contour -p -amax ELEV_MAX -amin ELEV_MIN -b 1 -i 1.0 -f "GPKG" /home/feromes/dev/fiacao-aerea/results/sample/MDS_3316-221-vegetation.tiff /home/feromes/dev/fiacao-aerea/results/sample/MDS_3316-221-vegetacao-countor.gpkg

0...10...20...30...40...50...60...70...80...90...100 - done.


In [22]:
gdf_countor = gpd.read_file('results/sample/MDS_3316-221-vegetacao-countor.gpkg')

In [28]:
gdf_countor = gpd.GeoDataFrame(gdf_countor.explode().reset_index())

/tmp/ipykernel_7067/2211291049.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gdf_countor = gpd.GeoDataFrame(gdf_countor.explode().reset_index())


In [29]:
gdf_fiacao = gpd.read_file('results/sample/fiacao-sample.gpkg')

In [34]:
gdf_fiacao.crs

<Derived Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [36]:
gdf_countor.set_crs(epsg=31983, inplace=True)

,level_0,level_1,ID,ELEV_MIN,ELEV_MAX,geometry
0,0,0,0,0.0,2.0,"POLYGON ((332237.670 7390943.990, 332238.170 7..."
1,0,1,0,0.0,2.0,"POLYGON ((332251.670 7390942.990, 332252.170 7..."
2,0,2,0,0.0,2.0,"POLYGON ((332437.670 7390942.990, 332438.170 7..."
3,0,3,0,0.0,2.0,"POLYGON ((332165.670 7390927.990, 332166.170 7..."
4,0,4,0,0.0,2.0,"POLYGON ((332214.670 7390941.990, 332215.170 7..."
...,...,...,...,...,...,...
540,0,540,0,0.0,2.0,"POLYGON ((332457.670 7390358.990, 332458.170 7..."
541,0,541,0,0.0,2.0,"POLYGON ((332491.670 7390358.990, 332492.170 7..."
542,0,542,0,0.0,2.0,"POLYGON ((332499.670 7390358.990, 332500.170 7..."
543,0,543,0,0.0,2.0,"POLYGON ((332544.670 7390358.990, 332545.170 7..."


In [39]:
gdf_countor.crs

<Derived Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [53]:
gdf_countor.geometry = gdf_countor.buffer(1)

In [55]:
gdf_countor_intersects = gdf_countor.sjoin(gdf_fiacao)

In [56]:
gdf_countor_intersects.geometry = gdf_countor_intersects.buffer(-1)

In [57]:
gdf_countor_intersects.to_file('results/sample/arvores_com_interseccao_com_rede.gpkg', driver='GPKG')

/home/feromes/miniconda3/envs/pdal/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
